# Advent of Code 2024 Day 9 

### Setup

In [1]:
from aocd import get_data, submit

day = 9
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
def parse_data(raw_data:str):
    return [int(x) for x in raw_data] 

sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

### Part One!

In [5]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

str(data)

In [7]:
from typing import Dict, List, Union, Literal

MemoryBlock = Union[int, Literal['.']]
Memory = List[MemoryBlock]

FileInfo = Dict[int, Dict[str, int]]
FreeSpaceInfo = Dict[int, List[int]]

StorageInfo = Dict[Union[Literal['freespace', 'files', 'max_file_size', 'max_freespace_size']], Union[Dict[int, FileInfo], FreeSpaceInfo]]


def is_freespace(block:MemoryBlock):
    return block == '.'

def is_filespace(block:MemoryBlock):
    return not is_freespace(block)


In [ ]:
def load_memory(discrete_representation: list[int]) -> Memory:
    memory = []

    for i in range(len(discrete_representation)):
        load_file = i % 2 == 0
        file_idx = i // 2
        block = file_idx if load_file else '.'
        memory += [block] * discrete_representation[i]
    
    return memory

load_memory(data)[:10]

In [9]:
import heapq

def load_storage_info(discrete_representation: list[int]) -> StorageInfo:
    storage_info: StorageInfo = {'files': {}, 'freespace': {}, 'max_file_size': 0, 'max_freespace_size': 0}
    offset = 0

    for i, block_size in enumerate(discrete_representation):
        load_file = i % 2 == 0
        idx = i // 2
        
        if load_file:
            storage_info['files'][idx] = {
                'file_location': offset,
                'file_size': block_size
            }

            if block_size > storage_info['max_file_size']:
                storage_info['max_file_size'] = block_size
        
        else:
            storage_info['freespace'].setdefault(block_size, [])
            heapq.heappush(storage_info['freespace'][block_size], offset)
            
            if block_size > storage_info['max_freespace_size']:
                storage_info['max_freespace_size'] = block_size

        offset += block_size
    
    return storage_info

In [10]:
def optimize_memory_with_file_fragmentation(memory: list[MemoryBlock], storage_info: dict[int: int]) ->  Memory:
    optimized_memory = memory[:]

    i, j = 0, len(memory) - 1

    while i < j:
        frontblock, endblock = optimized_memory[i], optimized_memory[j]

        if is_filespace(frontblock):
            i += 1
            continue

        elif is_freespace(endblock):
            j -= 1
            continue

        optimized_memory[i] = optimized_memory[j]
        optimized_memory[j] = '.'

        i += 1
        j -= 1

    return optimized_memory

In [11]:
def optimize_memory_without_file_fragmentation(memory: list[MemoryBlock], storage_info: StorageInfo) -> Memory:
    optimized_memory = memory[:]
    get_candidate_freespace_sizes = lambda size: ( k for k in storage_info['freespace'] if k >= size and len(storage_info['freespace'][k]) > 0 )

    for file, file_info in reversed(storage_info['files'].items()):
        file_size = file_info['file_size']
        file_location = file_info['file_location']

        for freespace_size in sorted(get_candidate_freespace_sizes(file_size), key=lambda k: storage_info['freespace'][k][0]):
            if freespace_size < file_size:
                continue
            
            if (storage_info['freespace'][freespace_size][0] >= file_location):
                continue

            idx = heapq.heappop(storage_info['freespace'][freespace_size])
            for offset in range(file_size):
                optimized_memory[idx + offset] = file
                optimized_memory[file_location + offset] = '.'
            
            heapq.heappush(storage_info['freespace'][file_size], file_location)
            
            if file_size != freespace_size:
                leftover_freespace = freespace_size - file_size
                leftover_idx = idx + file_size
                storage_info['freespace'].setdefault(leftover_freespace, [])
                heapq.heappush(storage_info['freespace'][leftover_freespace], leftover_idx)
            
            break
        
    return optimized_memory

In [12]:
def optimize_memory(memory: list[MemoryBlock], storage_info: dict[int: int], file_fragmentation=True) -> Memory:
    if file_fragmentation:
        return optimize_memory_with_file_fragmentation(memory=memory, storage_info=storage_info)
        
    else:
        return optimize_memory_without_file_fragmentation(memory=memory, storage_info=storage_info)


In [13]:
def generate_checksum(memory: list[MemoryBlock]) -> int:
    checksum = 0
    for i, block in enumerate(memory):
        checksum += block * i if is_filespace(block) else 0
    
    return checksum

In [ ]:
storage_info = load_storage_info(data)

memory = load_memory(data)

optimized_memory = optimize_memory(memory, storage_info)

checksum = generate_checksum(optimized_memory)

part_a_answer = checksum
part_a_answer

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [16]:
use_sample_data = False
part='b'

In [ ]:
data = sample_data if use_sample_data else test_data

str(data[:100]), len(data)

In [ ]:
storage_info = load_storage_info(data)

memory = load_memory(data)

optimized_memory = optimize_memory(memory, storage_info, file_fragmentation=False)

checksum = generate_checksum(optimized_memory)

part_b_answer = checksum
part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)

# Optimization Results 

The initial approach for Part 2's optimization without file fragmentation was let's say, less that optimal.... On the test data it was able to correctly optimize the memory but runs for a whopping 1.5 minutes. As such I spent some time optimizing the solution to make use of a tabulated set of freespace positions. Each size of freespace (0 - 9) has an associated heap. By compaing the top of the heap (peeking) of the freespace sizes that would fit the file we are able to quickly retrieve the location that the file is supposed to move to. This removes a linear scan of all the memory for each file and cuts down the runtime drastically. 

This section showcases the two times using %timeit for comparison 

The original un-optimal implementation. (it is clean tho)

In [20]:
def optimize_memory_without_file_fragmentation_original(memory: list[MemoryBlock], storage_info: dict[int: int]) -> list[MemoryBlock]:
    optimized_memory = memory[:]

    for file, file_info in reversed(storage_info['files'].items()):
        file_size = file_info['file_size']
        j = file_info['file_location']

        for i in range(len(optimized_memory)):
            if i >= j:
                break

            memory_slice = optimized_memory[i:i + file_size]
            is_freespace_available = all([ is_freespace(x) for x in memory_slice])

            if not is_freespace_available:
                continue

            for offset in range(file_size):
                optimized_memory[i + offset] = file
                optimized_memory[j + offset] = '.'
            
            break
    
    return optimized_memory

First the un-optimzied approach 

In [ ]:
%timeit optimize_memory_without_file_fragmentation_original(load_memory(data), load_storage_info(data))

59.5 s ± 286 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now the Optimized approach

In [24]:
%timeit optimize_memory_without_file_fragmentation(load_memory(data), load_storage_info(data))

18.9 ms ± 150 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
